# Jax jit

## Goals:

- Using `jit`

- Understanding **when** to use jit a.k.a why not jit everything?

## Concepts:

- jit
- fori-loop
- functional programming

In [9]:
import numpy as np
from typing import TypeAlias
import time

from hyperparameters import (
    _dt,
    _t_max,
    _tau_m,
    _V_reset,
    _V_thresh,
    _R,
    num_simulations
)
import jax.numpy as jnp
import jax
from tqdm.notebook import tqdm

with open('weights.npy', 'rb') as f:
    W = np.load(f)

# Initial conditions
n_neurons = len(W)# Number of neurons in the network
_V = jnp.ones(n_neurons) * _V_reset  # Initial potentials

# Type Definitions for Clarity

In [10]:
Tensor3D: TypeAlias = jnp.ndarray
Mat: TypeAlias = jnp.ndarray
Vec: TypeAlias = jnp.ndarray 

# Run the Simulations

In [11]:
def run_simulation(
    W: Mat,
    V: Vec,

    # Neuron Parameters
    tau_m: float,
    v_reset: float,
    v_thresh: float,
    membr_R: float,

    # How long do we run for? 
    t_max: float,
    dt: float, 

):
    # Simulation

    spike_train = []
    for i, t in enumerate(np.arange(0, t_max, dt)):
        if i == 0:
            continue
        V, spike = run_step(V, v_thresh, v_reset, W, tau_m, dt, membr_R)
        spike_train.append(spike)

    return spike_train

In [12]:
@jax.jit
def run_step(
    v_prev,
    v_thresh,
    v_reset,

    W,
    tau_m,
    dt,
    membr_R,
):
    spiked = v_prev >= v_thresh
    V = jnp.where(spiked, v_reset, v_prev)

    # Update voltages
    I_syn = W.dot(spiked)  # Synaptic current from spikes
    dV = (dt / tau_m) * (-V + v_reset + membr_R * I_syn)
    V = V + dV

    # No self-inputs; neurons cannot spike themselves in this timestep
    V = jnp.where(spiked, v_reset, V)
    return V, spiked



In [5]:
time_arr = []
for i in range(num_simulations):
    start = time.time()
    spike_train = run_simulation(
        W,
        _V,
        _tau_m, _V_reset, _V_thresh, _R,
        _t_max, _dt
    )
    end = time.time()
    print(f"Iteration {i} took: {end - start} seconds")
    time_arr.append(end - start)
    if i > 2:
        print("Breaking out - point proven")
        break

print(f"Average Time: {np.mean(time_arr)}")
print(f"S.Dev Time: {np.std(time_arr)}")

Iteration 0 took: 41.892136335372925 seconds


KeyboardInterrupt: 

# What gives? 

`jax.jit` doesn't play nicely with numpy! There are times where calling `jnp.asarray` is necessary

In [13]:
time_arr = []
for i in range(num_simulations):
    start = time.time()
    spike_train = run_simulation(
        jnp.asarray(W),
        _V,
        _tau_m, _V_reset, _V_thresh, _R,
        _t_max, _dt
    )
    end = time.time()
    # print(f"Iteration {i} took: {end - start} seconds")
    time_arr.append(end - start)

print(f"Average Time: {np.mean(time_arr)}")
print(f"S.Dev Time: {np.std(time_arr)}")

Average Time: 0.6576376676559448
S.Dev Time: 0.05285802814637119


# `fori_loop`

In [14]:
from functools import partial

@jax.jit
def run_step(
    v_prev,
    v_thresh,
    v_reset,

    W,
    tau_m,
    dt,
    membr_R,
):
    spiked = v_prev >= v_thresh
    V = jnp.where(spiked, v_reset, v_prev)

    # Update voltages
    I_syn = W.dot(spiked)  # Synaptic current from spikes
    dV = (dt / tau_m) * (-V + v_reset + membr_R * I_syn)
    V = V + dV

    # No self-inputs; neurons cannot spike themselves in this timestep
    V = jnp.where(spiked, v_reset, V)
    return V, spiked

def run_simulation(W, V, tau_m, v_reset, v_thresh, membr_R, t_max, dt, num_steps):
    def body_func(i, val):
        V, spike_train = val
        V, spike = run_step(V, v_thresh, v_reset, W, tau_m, dt, membr_R)
        return (V, spike_train.at[i].set(spike))


    spike_train = jnp.zeros((num_steps, len(W)), dtype=bool)
    _, spike_train = jax.lax.fori_loop(0, num_steps, body_func, (V, spike_train))
    return spike_train

In [15]:
time_arr = []
for i in range(num_simulations):
    start = time.time()
    spike_train = run_simulation(
        jnp.asarray(W),
        _V,
        _tau_m, _V_reset, _V_thresh, _R,
        _t_max, _dt, num_steps = int(_t_max / _dt)
    )
    end = time.time()
    print(f"Iteration {i} took: {end - start} seconds")
    time_arr.append(end - start)

print(f"Average Time: {np.mean(time_arr)}")
print(f"S.Dev Time: {np.std(time_arr)}")

Iteration 0 took: 0.21049213409423828 seconds
Iteration 1 took: 0.14664840698242188 seconds
Iteration 2 took: 0.14410853385925293 seconds
Iteration 3 took: 0.1443476676940918 seconds
Iteration 4 took: 0.141160249710083 seconds
Iteration 5 took: 0.13787245750427246 seconds
Iteration 6 took: 0.1391611099243164 seconds
Iteration 7 took: 0.13631343841552734 seconds
Iteration 8 took: 0.13673853874206543 seconds
Iteration 9 took: 0.1355299949645996 seconds
Average Time: 0.14723725318908693
S.Dev Time: 0.021391894203354116
